In [1]:
import gc
import glob
import numpy as np
import os
import pandas as pd
import pickle
import random
import time
from tqdm import tqdm

from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import xgboost as xgb

#import tensorflow as tf
#import keras
#from keras import losses, regularizers
#from keras.models import Sequential
#from keras.layers import Dense, Dropout
#from keras import backend as K

from sklearn.linear_model import ElasticNet

### Input

Classifiers

In [2]:
classifiers = ['ElasticNet','GBM']

Hyperopt parameters

In [3]:
n_hyperopt_iterations = 2**8

Data splits

In [4]:
n_splits_trainvalidation_test = 20
test_size = 0.2
k_train_validation = 5
early_stopping_size = 0.125

Seed

In [5]:
seed_ = 1

# implement seed
random.seed(seed_)
np.random.seed(seed_)

### HyperOpt Functions

In [6]:
def hyperopt_function(parameters):

    # load data
    with open('_files/data.pickle', 'rb') as f:
        train_data, X_validation, y_validation = pickle.load(f, encoding='latin1')
    
    # calculate performance
    mean_validation_mse = hyperopt_performance(train_data, X_validation, y_validation, parameters)
    gc.collect()
    
    # save validation predictions if best classifier
    with open('_files/validation.pickle','rb') as f:
        best_mse = pickle.load(f)
    if mean_validation_mse < best_mse:
        with open('_files/validation.pickle','wb') as f:
            pickle.dump(mean_validation_mse, f)
    
    # return performance
    return {'loss':mean_validation_mse, 'status':STATUS_OK}

In [7]:
def hyperopt_performance(train_data, X_validation, y_validation, parameters):
    
    # unpack train data
    if classifiers[a] in ['GBM','NeuralNet']:
        X_training_train, y_training_train, X_earlystopping_train, y_earlystopping_train = train_data[0], train_data[1], train_data[2], train_data[3]
    else:
        X_train, y_train = train_data[0], train_data[1]
    
    # initialize validation performance and predictions
    validation_mse = []
    
    # iterate over number of training/validation splits
    for i in range(k_train_validation):
        
        # elastic net
        if classifiers[a] == 'ElasticNet':
            
            # create classifier
            clf = ElasticNet(max_iter=1000000, random_state=seed_, **parameters)
            
            # train on training
            clf.fit(X_train[i], y_train[i])
            
            # evaluate on validation
            y_pred = clf.predict(X_validation[i])
            mse = mean_squared_error(y_validation[i], y_pred)
            validation_mse.append(mse)
            
        # GBM
        elif classifiers[a] == 'GBM':

            # xgb datasets
            xgb_training = xgb.DMatrix(X_training_train[i], label=y_training_train[i])
            xgb_earlystopping = xgb.DMatrix(X_earlystopping_train[i], label=y_earlystopping_train[i])
            xgb_validation = xgb.DMatrix(X_validation[i], label=y_validation[i])

            # parameters
            param = parameters.copy()
            param['objective'] = 'reg:squarederror'
            param['eval_metric'] = 'rmse'
            param['seed'] = seed_
            evallist = [(xgb_training, 'train'), (xgb_earlystopping, 'eval')]

            # train on training
            bst = xgb.train(param, xgb_training, num_boost_round=10000, evals=evallist, early_stopping_rounds=10, verbose_eval=False)

            # evaluate on validation
            y_pred = bst.predict(xgb_validation, ntree_limit=bst.best_ntree_limit)
            mse = mean_squared_error(y_validation[i], y_pred)
            validation_mse.append(mse)
            
        # neural network
        elif classifiers[a] == 'NeuralNet':
            
            # train best model on training+validation set
            with tf.Graph().as_default():
                with tf.Session() as sess:

                    # create model
                    model = Sequential()
                    for j in range(parameters['number_of_layers']):
                        model.add(Dense(parameters['neurons_per_layer'], activation=parameters['activation_function']))
                        model.add(Dropout(parameters['dropout_rate']))
                    model.add(Dense(1))

                    # loss and performance metrics
                    model.compile(loss='mean_squared_error', optimizer=parameters['optimizer'])
                    earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto', restore_best_weights=True)

                    # fit and evaluate model
                    model.fit(X_training_train[i], y_training_train[i], epochs=1000, verbose=0, validation_data=(X_earlystopping_train[i], y_earlystopping_train[i]), callbacks=[earlystopping])
                    y_pred = model.predict(X_validation[i], batch_size=len(y_validation[i]), verbose=0)
                    mse = mean_squared_error(y_validation[i], y_pred)
                    validation_mse.append(mse)

            # clear session
            K.clear_session()

    # average validation performance over all folds
    mean_validation_mse = np.mean(validation_mse) + np.std(validation_mse)/np.sqrt(len(validation_mse))
    return mean_validation_mse

### Load gene lists

In [8]:
# gene lists
with open('../gene_lists/gene_lists.pickle','rb') as f:
    genelists, genes = pickle.load(f)

### Pipeline

In [9]:
# iterate over classifiers
for a in range(len(classifiers)):
    
    # output folder
    if not os.path.isdir(classifiers[a]):
        os.mkdir(classifiers[a])
    
    # performance files
    performance_mse = pd.DataFrame(index=['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)]+['MEAN','STERR'], columns=genelists)
    performance_mse.to_csv('%s/mse.csv' % classifiers[a])

    performance_mae = pd.DataFrame(index=['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)]+['MEAN','STERR'], columns=genelists)
    performance_mae.to_csv('%s/mae.csv' % classifiers[a])

    performance_explainedvariance = pd.DataFrame(index=['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)]+['MEAN','STERR'], columns=genelists)
    performance_explainedvariance.to_csv('%s/explainedvariance.csv' % classifiers[a])
    
    performance_r2 = pd.DataFrame(index=['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)]+['MEAN','STERR'], columns=genelists)
    performance_r2.to_csv('%s/r2.csv' % classifiers[a])
    
    # iterate over gene lists
    for b in range(len(genelists)):
        print('-------------------------')
        print('CLASSIFIER: %s' % classifiers[a])
        print('GENE LIST: %s' % genelists[b])
        print('-------------------------')

        # load dataset
        with open('../CCLE/CCLE.pickle', 'rb') as f:
            X_matrix, y_vector = pickle.load(f, encoding='latin1')
        y_vector = np.array(y_vector)
        
        # subset features
        X_matrix = X_matrix[genes[b]]
        
        # divide train+validation from testing
        trainvalidation_index = []
        test_index = []
        sss = ShuffleSplit(n_splits=n_splits_trainvalidation_test, test_size=test_size, random_state=seed_)
        for trainvalidation_, test_ in sss.split(X_matrix):
            trainvalidation_index.append(list(trainvalidation_))
            test_index.append(list(test_))

        # iterate over number of training+validation/testing splits
        for c in range(n_splits_trainvalidation_test):
            print('Split %d' % (c+1))

            # separate train+validation and testing
            X_trainvalidation = X_matrix.iloc[trainvalidation_index[c],]
            X_test = X_matrix.iloc[test_index[c],]
            y_trainvalidation = y_vector[trainvalidation_index[c]]
            y_test = y_vector[test_index[c]]

            # separate training_trainvalidation from earlystopping_trainvalidation
            if classifiers[a] in ['GBM','NeuralNet']:
                training_index = []
                earlystopping_index = []
                sss = ShuffleSplit(n_splits=1, test_size=early_stopping_size, random_state=seed_)
                for training_, earlystopping_ in sss.split(X_trainvalidation):
                    training_index.append(list(training_))
                    earlystopping_index.append(list(earlystopping_))
                X_training_trainvalidation = X_trainvalidation.iloc[training_index[0],]
                X_earlystopping_trainvalidation = X_trainvalidation.iloc[earlystopping_index[0],]
                y_training_trainvalidation = y_trainvalidation[training_index[0]]
                y_earlystopping_trainvalidation = y_trainvalidation[earlystopping_index[0]]

            # divide train from validation
            train_index = []
            validation_index = []
            skf = KFold(n_splits=k_train_validation, shuffle=True, random_state=seed_)
            for train_, validation_ in skf.split(X_trainvalidation):
                train_index.append(list(train_))
                validation_index.append(list(validation_))

            # separate train and validation
            X_train = []
            X_validation = []
            y_train = []
            y_validation = []
            for d in range(k_train_validation):
                X_train.append(X_trainvalidation.iloc[train_index[d],])
                X_validation.append(X_trainvalidation.iloc[validation_index[d],])
                y_train.append(y_trainvalidation[train_index[d]])
                y_validation.append(y_trainvalidation[validation_index[d]])

            # separate training_train from earlystopping_train
            if classifiers[a] in ['GBM','NeuralNet']:
                X_training_train = []
                X_earlystopping_train = []
                y_training_train = []
                y_earlystopping_train = []
                for d in range(k_train_validation):
                    training_index = []
                    earlystopping_index = []
                    sss = ShuffleSplit(n_splits=1, test_size=early_stopping_size, random_state=seed_)
                    for training_, earlystopping_ in sss.split(X_train[d]):
                        training_index.append(list(training_))
                        earlystopping_index.append(list(earlystopping_))
                    X_training_train.append(X_train[d].iloc[training_index[0],])
                    X_earlystopping_train.append(X_train[d].iloc[earlystopping_index[0],])
                    y_training_train.append(y_train[d][training_index[0]])
                    y_earlystopping_train.append(y_train[d][earlystopping_index[0]])
            
            # imputation and scaling
            if classifiers[a] in ['ElasticNet','NeuralNet']:
                
                # train+validation, testing
                imp = SimpleImputer(missing_values=np.nan, strategy='mean')
                X_trainvalidation = imp.fit_transform(X_trainvalidation)
                X_test = imp.transform(X_test)
                scaler = StandardScaler()
                X_trainvalidation = scaler.fit_transform(X_trainvalidation)
                X_test = scaler.transform(X_test)
                
                # training_trainvalidation, earlystopping_trainvalidation
                if classifiers[a] in ['GBM','NeuralNet']:
                    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
                    X_training_trainvalidation = imp.fit_transform(X_training_trainvalidation)
                    X_earlystopping_trainvalidation = imp.transform(X_earlystopping_trainvalidation)
                    scaler = StandardScaler()
                    X_training_trainvalidation = scaler.fit_transform(X_training_trainvalidation)
                    X_earlystopping_trainvalidation = scaler.transform(X_earlystopping_trainvalidation)
                    
                # train, validation
                for d in range(k_train_validation):
                    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
                    X_train[d] = imp.fit_transform(X_train[d])
                    X_validation[d] = imp.transform(X_validation[d])
                    scaler = StandardScaler()
                    X_train[d] = scaler.fit_transform(X_train[d])
                    X_validation[d] = scaler.transform(X_validation[d])
                    
                # training_train, earlystopping_train
                if classifiers[a] in ['GBM','NeuralNet']:
                    for d in range(k_train_validation):
                        imp = SimpleImputer(missing_values=np.nan, strategy='mean')
                        X_training_train[d] = imp.fit_transform(X_training_train[d])
                        X_earlystopping_train[d] = imp.transform(X_earlystopping_train[d])
                        scaler = StandardScaler()
                        X_training_train[d] = scaler.fit_transform(X_training_train[d])
                        X_earlystopping_train[d] = scaler.transform(X_earlystopping_train[d])

            # hyperopt parameters
            if classifiers[a] == 'ElasticNet':
                parameters = {
                    'alpha':hp.loguniform('alpha',np.log(1e-5),np.log(1e0)),
                    'l1_ratio': hp.uniform('l1_ratio', 0, 1)
                }
            elif classifiers[a] == 'GBM':
                parameters = {
                    'gamma': hp.loguniform('gamma', np.log(0.0001), np.log(5)) - 0.0001,
                    'max_depth': scope.int(hp.uniform('max_depth', 1, 11)),
                    'subsample': hp.uniform('subsample', 0.5, 1),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
                    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
                    'reg_lambda': hp.loguniform('reg_lambda', np.log(1), np.log(4)),
                    'reg_alpha': hp.loguniform('reg_alpha', np.log(0.0001), np.log(1)) - 0.0001,
                    'eta': hp.loguniform('eta', np.log(0.01), np.log(0.5))
                }
            elif classifiers[a] == 'NeuralNet':
                parameters_choice = {
                    'activation_function':['relu','elu','sigmoid'],
                    'optimizer':['rmsprop', 'adam', 'sgd']
                }
                parameters = {
                    'number_of_layers':scope.int(hp.quniform('number_of_layers',1.5,5.5,1)),
                    'neurons_per_layer':scope.int(hp.qloguniform('neurons_per_layer',np.log(10.5),np.log(100.5),1)),
                    'activation_function':hp.choice('activation_function', ['relu','elu','sigmoid']),
                    'dropout_rate':hp.uniform('dropout_rate',0,0.5),
                    'optimizer':hp.choice('optimizer', ['rmsprop', 'adam', 'sgd'])
                }

            # save info for hyperopt
            with open('_files/validation.pickle','wb') as f:
                pickle.dump(1000., f)
            with open('_files/data.pickle','wb') as f:
                if classifiers[a] in ['GBM','NeuralNet']:
                    pickle.dump([[X_training_train, y_training_train, X_earlystopping_train, y_earlystopping_train], X_validation, y_validation], f)
                else:
                    pickle.dump([[X_train, y_train], X_validation, y_validation], f)
        
            # hyperopt to find best parameters
            trials = Trials()
            best = fmin(hyperopt_function, parameters, algo=tpe.suggest, max_evals=n_hyperopt_iterations, trials=trials, rstate=np.random.RandomState(seed_), verbose=0, show_progressbar=True)
            
            # elastic net
            if classifiers[a] == 'ElasticNet':
                
                # parameters
                parameters = {
                    'alpha':best['alpha'],
                    'l1_ratio':best['l1_ratio']
                }
                
                # create classifier
                clf = ElasticNet(max_iter=1000000, random_state=seed_, **parameters)

                # train on training+validation
                clf.fit(X_trainvalidation, y_trainvalidation)

                # evaluate on testing
                y_pred = clf.predict(X_test)

            # GBM
            elif classifiers[a] == 'GBM':
                
                # parameters
                parameters = {
                    'gamma': best['gamma'],
                    'max_depth': int(np.round(best['max_depth'])),
                    'subsample': best['subsample'],
                    'colsample_bytree': best['colsample_bytree'],
                    'colsample_bylevel': best['colsample_bylevel'],
                    'reg_lambda': best['reg_lambda'],
                    'reg_alpha': best['reg_alpha'],
                    'eta':best['eta']
                }
                
                # xgb datasets
                xgb_training = xgb.DMatrix(X_training_trainvalidation, label=y_training_trainvalidation)
                xgb_earlystopping = xgb.DMatrix(X_earlystopping_trainvalidation, label=y_earlystopping_trainvalidation)
                xgb_testing = xgb.DMatrix(X_test, label=y_test)

                # parameters
                param = parameters.copy()
                param['objective'] = 'reg:squarederror'
                param['eval_metric'] = 'rmse'
                param['seed'] = seed_
                evallist = [(xgb_training, 'train'), (xgb_earlystopping, 'eval')]

                # train on training
                bst = xgb.train(param, xgb_training, num_boost_round=10000, evals=evallist, early_stopping_rounds=10, verbose_eval=False)

                # evaluate on validation
                y_pred = bst.predict(xgb_testing, ntree_limit=bst.best_ntree_limit)

            # neural network
            elif classifiers[a] == 'NeuralNet':
                
                # train best model on training+validation set
                with tf.Graph().as_default():
                    with tf.Session() as sess:

                        # create model
                        model = Sequential()
                        for j in range(int(np.round(best['number_of_layers']))):
                            model.add(Dense(int(np.round(best['neurons_per_layer'])), activation=parameters_choice['activation_function'][best['activation_function']]))
                            model.add(Dropout(best['dropout_rate']))
                        model.add(Dense(1))

                        # loss and performance metrics
                        model.compile(loss='mean_squared_error', optimizer=parameters_choice['optimizer'][best['optimizer']])      
                        earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto', restore_best_weights=True)

                        # fit and evaluate model
                        model.fit(X_training_trainvalidation, y_training_trainvalidation, epochs=1000, verbose=0, validation_data=(X_earlystopping_trainvalidation, y_earlystopping_trainvalidation), callbacks=[earlystopping])
                        y_pred = model.predict(X_test, batch_size=len(y_test), verbose=0)

                # clear session
                K.clear_session()

            # calculate test performance - mse
            performance = mean_squared_error(y_test, y_pred)
            performance_mse.at['split_%d' % (c+1), genelists[b]] = performance
            performance_mse.at['MEAN', genelists[b]] = np.nanmean(performance_mse.loc[['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)], genelists[b]].values.tolist())
            performance_mse.at['STERR', genelists[b]] = np.nanstd(performance_mse.loc[['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)], genelists[b]].values.tolist())/np.sqrt(c+1)
            performance_mse.to_csv('%s/mse.csv' % classifiers[a])

            # calculate test performance - mae
            performance = mean_absolute_error(y_test, y_pred)
            performance_mae.at['split_%d' % (c+1), genelists[b]] = performance
            performance_mae.at['MEAN', genelists[b]] = np.nanmean(performance_mae.loc[['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)], genelists[b]].values.tolist())
            performance_mae.at['STERR', genelists[b]] = np.nanstd(performance_mae.loc[['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)], genelists[b]].values.tolist())/np.sqrt(c+1)
            performance_mae.to_csv('%s/mae.csv' % classifiers[a])

            # calculate test performance - explained variance
            performance = explained_variance_score(y_test, y_pred)
            performance_explainedvariance.at['split_%d' % (c+1), genelists[b]] = performance
            performance_explainedvariance.at['MEAN', genelists[b]] = np.nanmean(performance_explainedvariance.loc[['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)], genelists[b]].values.tolist())
            performance_explainedvariance.at['STERR', genelists[b]] = np.nanstd(performance_explainedvariance.loc[['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)], genelists[b]].values.tolist())/np.sqrt(c+1)
            performance_explainedvariance.to_csv('%s/explainedvariance.csv' % classifiers[a])
            
            # calculate test performance - r2
            performance = r2_score(y_test, y_pred)
            performance_r2.at['split_%d' % (c+1), genelists[b]] = performance
            performance_r2.at['MEAN', genelists[b]] = np.nanmean(performance_r2.loc[['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)], genelists[b]].values.tolist())
            performance_r2.at['STERR', genelists[b]] = np.nanstd(performance_r2.loc[['split_%d' % x for x in range(1,n_splits_trainvalidation_test+1)], genelists[b]].values.tolist())/np.sqrt(c+1)
            performance_r2.to_csv('%s/r2.csv' % classifiers[a])

-------------------------
CLASSIFIER: ElasticNet
GENE LIST: Lewis
-------------------------
Split 1
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [06:18<00:00,  1.67s/it, best loss: 1.3418591254280121]
Split 2
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [09:33<00:00,  2.24s/it, best loss: 1.1599947232504353]
Split 3
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [10:11<00:00,  4.57s/it, best loss: 1.1937186505540267]
Split 4
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0911717906625138, tolerance: 0.043164068273432846
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06099582014426402, tolerance: 0.04370547549373135
  positive)



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [51:06<00:00, 35.15s/it, best loss: 1.1009995628423865]
Split 10
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [08:27<00:00,  1.10it/s, best loss: 1.1558886633284344]
Split 11
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [15:35<00:00,  1.46s/it, best loss: 1.1386499674770116]
Split 12
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [10:31<00:00,  2.45s/it, best loss: 1.0995379646399912]
Split 13
100%|███████████

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.15728248014630733, tolerance: 0.04290048568059701
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.18471889724413068, tolerance: 0.04152481737432836
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11088387383490306, tolerance: 0.041755626962388055
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_mod

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [1:05:55<00:00,  1.71it/s, best loss: 1.0997219581569901]
Split 4
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [13:26<00:00,  3.15s/it, best loss: 1.1489104374198968]
Split 5
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [15:25<00:00, 24.24s/it, best loss: 1.112676650039293]
Split 6
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [36:30<00:00, 12.19s/it, best loss: 1.1796247607460864]
Split 7
100%|███████████████

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [41:49<00:00,  8.61s/it, best loss: 1.1989871762603006]
Split 13
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [19:50<00:00,  4.65s/it, best loss: 1.2334882298074212]
Split 14
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [39:24<00:00,  8.37s/it, best loss: 1.0724349858234679]
Split 15
 51%|█████████████████████████████████████████████████████████████████████████████████████▍                                                                                 | 131/256 [10:24<05:52,  2.82s/it, best loss: 1.158112306561023]

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0921413029620766, tolerance: 0.04358590748059703
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09083521821254108, tolerance: 0.043241781943283576
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.10032996487084485, tolerance: 0.04262878769970149
  positive)



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [40:57<00:00, 12.94s/it, best loss: 1.1315383978993308]
Split 16
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [14:01<00:00,  6.82s/it, best loss: 1.243894884880558]
Split 17
 43%|███████████████████████████████████████████████████████████████████████▎                                                                                              | 110/256 [11:10<11:17,  4.64s/it, best loss: 1.1777030244793922]

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.16893126578862802, tolerance: 0.04377580429432836
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.15365125556830345, tolerance: 0.04288585196537314
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.14294549636591297, tolerance: 0.04276050976597015
  positive)

c:\users\jlewis43\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_mode

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [44:32<00:00, 10.44s/it, best loss: 1.167534568445505]
Split 18
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [10:17<00:00,  2.41s/it, best loss: 1.1864517269520953]
Split 19
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [13:17<00:00,  3.12s/it, best loss: 1.234423421833014]
Split 20
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [10:33<00:00,  1.40s/it, best loss: 1.1535394580117475]
-------------------------